# Bringing out brain regions

## Description

This tool allows you to **hightlight one or more regions of interest in the mouse brain** and visualize them in coronal, sagittal or transversal section.    
The mouse brain reference volumes are a pair of volumetric files from the Allen Institute for Brain Science (AIBS): the Nissl and the annotations volumes. 
More specifically:   
* the **Nissl volume** shows every cell in the brain and it is used to obtain the cell distribution
* the **annotation volume** provides a brain region ID for each voxel of the Nissl volume and any volumetric dataset of the AIBS. 

The mouse brain regions are organized in a hierarchical tree and this **hierarchy** is stored in a **json file**, common to each reference atlas version (it is also available on the AIBS website). In this hierarchical tree every region is divided in subregions wich, in turn, are divided in other subregions and so on, with the leaves of the tree representing the finest subdivisions captured by the AIBS.  


Here are some preliminary steps: this tool needs some modules to work properly, so you need to import them first

In [ ]:
import numpy as np
from voxcell import RegionMap, VoxelData
from os.path import join
from matplotlib.pylab import plt
import ipywidgets


Set below (**DATA_FOLDER**) the path to the folder containing the data, including the annotations volumes, the Nissl volumes and the json hierarchy file.

Set also the folder in which the figures produced will be stored (**OUTPUT_FOLDER**).


In [ ]:
DATA_FOLDER = '../data/'
OUTPUT_FOLDER = "../output"

Now, load the following files: 
* the json file containing the hierarchy of the brain regions
* the annotation volume file 
* the nissl volume file 


In [ ]:
region_map = RegionMap.load_json(join(DATA_FOLDER, "1.json"))
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, "bbp/annotation_25.nrrd")).raw
nissl = VoxelData.load_nrrd(join(DATA_FOLDER, "bbp/ara_nissl_25.nrrd")).raw

In the following section, create a function to convert a Hexadecimal color into its RGB value counterpart; you will need this later

In [ ]:
def hex_to_rgb(value):
    """
    Converts a Hexadecimal color into its RGB value counterpart.

    Arguments:
        value: string hexadecimal color to convert.
    Returns:
        List of the Red, Green, and Blue components of the color
    """

    value = value.lstrip('#')
    lv = len(value)
    return np.array([int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)])


## Parameters for the plot

Here is the interesting part: you can write down one or more regions that you want to highlight!  

*Be carefull*: the name of the region has to be written in the same way as it is in the json file, but it not necessarily has to be the full name 

In [ ]:
regions_to_highlight = ["raphe nuclei"]

### Filtering the regions of interest

In the following part, the algorithm picks up the IDs of the selected regions (in *"region_to_highlight"*) and their descendants. 

In [ ]:
ids_region = []
for region_name in regions_to_highlight:
    ids_roi=list(region_map.find("@.*{}.*$".format(region_name), 
                                           attr="name", with_descendants=True, 
                                           ignore_case=True))

#if the name does not correspond to an existing region in the json file, an error message will be displayed 
    if len(ids_roi)==0:
        print("WARNING: No region found for: {}".format(region_name))  
        continue
    ids_region.append(ids_roi)

if len(ids_region)==0:
    print("WARNING: No region selected, please select at least one valid region")

Now, it assigns a color for each voxel in the Annotation volume corresponding to the IDs in *region_to_highlight*

In [ ]:
colored_annotations = np.full(annotation.shape + (4,), 1.)
colored_annotations[:,:,:,3] = 0.
first_pos = np.zeros(3) #starting position of the slice for the figure ("default slice")
max_counts = np.zeros(3) #count maximum number of voxel in slice corresponding to the regions of interest 

for id_reg in ids_region:
    ann_filt = np.isin(annotation, id_reg) 
    color = hex_to_rgb(region_map.get(id_reg[0], attr="color_hex_triplet")) / 255. #get the color of one of the regions of the list (the regions of interest and the subregions). 
    colored_annotations[ann_filt, :3] = color #rgb color of the region 
    colored_annotations[ann_filt, 3] = 0.5 #transparence 

    # the following lines ensure that the default slice (pictured in the next section) contains the maximum number of voxel of the selected regions:
    for i, dim in enumerate(np.where(ann_filt)):
        u, c = np.unique(dim, return_counts=True)
        argmax_c = np.argmax(c) 
        max_c = c[argmax_c]
        if max_c > max_counts[i]:
            first_pos[i] = u[argmax_c]
            max_counts[i] = max_c

## Visualize brain slices

Now you can display brain slices with the selected region highlighted!
After running the following lines, you will see the the default *coronal slice*, containing the maximum number of voxel corrisponding to the selected regions, with these regions colored. You can choose the desired slice from 1 to 319 mooving the cursor back and forth, holding down the left mouse button until you reach it. You can choose to display:
* **sagittal slices**: selecting 2 in the "direction" toolbar above the slice 
* **coronal slices**: selecting 0 in the toolbar
* **transversal slices**: selecting 1 in the toolbar

In [ ]:
cmap = plt.cm.gray  # color map for Nissl
@ipywidgets.interact
def f_final_outer(direction=ipywidgets.Dropdown(options=[0, 1, 2])):
    @ipywidgets.interact
    def f_final_inner(i=ipywidgets.IntSlider(value=first_pos[direction],
                                             min=0, max=annotation.shape[direction] - 1, 
                                             continuous_update=False)):
        fig, ax = plt.subplots(figsize=(15, 9.1))
        plt.suptitle('Section number: {}'.format(i))
        if direction ==2:
            ax.imshow(np.take(1-nissl, i, axis=direction).T, cmap=cmap, interpolation="nearest")
            ax.imshow(np.moveaxis(np.take(colored_annotations, i, axis=direction), 0, 1), interpolation="nearest")
        else:
            ax.imshow(np.take(1-nissl, i, axis=direction), cmap=cmap, interpolation="nearest")
            ax.imshow(np.take(colored_annotations, i, axis=direction), interpolation="nearest")
        ax.axis('off')